In [2]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

In [22]:
!pip install PyQt5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 14.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.0 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.7/346.7 kB 15.7 MB/s eta 0:00:00


In [20]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt
%matplotlib notebook

In [4]:
DATASET_PATH = f"{os.getcwd()}/data/hand_signs"
TRAIN_PATH = f"{DATASET_PATH}/train"
TEST_PATH = f"{DATASET_PATH}/test"

In [5]:
print(DATASET_PATH)
labels = []
for i in os.listdir(TRAIN_PATH):
  if os.path.isdir(os.path.join(TRAIN_PATH, i)):
    labels.append(i)
print(labels)

/home/steffen/Projects/Private/hackathon/chakra_lock/data/hand_signs
['boar', 'ram', 'bird', 'hare', 'monkey', 'zero', 'horse', 'snake', 'tiger', 'dragon', 'dog', 'none', 'ox', 'rat']


In [25]:
NUM_EXAMPLES = 1

for label in labels:
    label_dir = os.path.join(TRAIN_PATH, label)
    example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
    fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
    for i in range(NUM_EXAMPLES):
        axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
        axs[i].get_xaxis().set_visible(False)
        axs[i].get_yaxis().set_visible(False)
    fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')
plt.show()

<IPython.core.display.Javascript object>

TypeError: 'Axes' object is not subscriptable

In [26]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=TRAIN_PATH,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)

INFO:tensorflow:Loading image /home/steffen/Projects/Private/hackathon/chakra_lock/data/hand_signs/train/monkey/monkey_IMG_31a354754-4aa2-11ea-8a97-0242ac1c0002.jpg
INFO:tensorflow:Loading image /home/steffen/Projects/Private/hackathon/chakra_lock/data/hand_signs/train/boar/IMG_e8800a40-547b-11ea-bfdc-48f17fc25591.png


I0000 00:00:1705755058.197234   13023 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1705755058.203355   30064 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.3.3-arch1.1), renderer: AMD Radeon Graphics (radeonsi, renoir, LLVM 16.0.6, DRM 3.56, 6.7.0-arch3-1)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


INFO:tensorflow:Loading image /home/steffen/Projects/Private/hackathon/chakra_lock/data/hand_signs/train/horse/horse_IMG_048e77876-4aa0-11ea-a68b-0242ac1c0002.jpg
INFO:tensorflow:Loading image /home/steffen/Projects/Private/hackathon/chakra_lock/data/hand_signs/train/horse/IMG_e857668a-547b-11ea-960d-48f17fc25591.png
INFO:tensorflow:Loading image /home/steffen/Projects/Private/hackathon/chakra_lock/data/hand_signs/train/ox/ox_IMG_289ee99e8-46b4-11ea-a083-0242ac1c0002.jpg
INFO:tensorflow:Loading image /home/steffen/Projects/Private/hackathon/chakra_lock/data/hand_signs/train/dog/IMG_e8436ac8-547b-11ea-8881-48f17fc25591.png
INFO:tensorflow:Loading image /home/steffen/Projects/Private/hackathon/chakra_lock/data/hand_signs/train/dragon/dragon_IMG_4617065ac-4a9f-11ea-a68b-0242ac1c0002.jpg
INFO:tensorflow:Loading image /home/steffen/Projects/Private/hackathon/chakra_lock/data/hand_signs/train/hare/IMG_e88ab543-547b-11ea-90b4-48f17fc25591.png
INFO:tensorflow:Loading image /home/steffen/Projec

INFO:tensorflow:Load valid hands with size: 1922, num_label: 14, labels: none,bird,boar,dog,dragon,hare,horse,monkey,ox,ram,rat,snake,tiger,zero.


In [27]:
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [28]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 14)                1806      
 out (Dense)                                                     
                                                             

INFO:tensorflow:Training the models...


Resuming from exported_model/epoch_models/model-0010
Epoch 1/10
768/768 [==============================] - 3s 3ms/step - loss: 0.7293 - categorical_accuracy: 0.6979 - val_loss: 0.2243 - val_categorical_accuracy: 0.9115 - lr: 0.0010
Epoch 2/10
768/768 [==============================] - 2s 2ms/step - loss: 0.6851 - categorical_accuracy: 0.7233 - val_loss: 0.2266 - val_categorical_accuracy: 0.9062 - lr: 9.9000e-04
Epoch 3/10
768/768 [==============================] - 2s 2ms/step - loss: 0.6542 - categorical_accuracy: 0.7298 - val_loss: 0.2258 - val_categorical_accuracy: 0.9062 - lr: 9.8010e-04
Epoch 4/10
768/768 [==============================] - 2s 2ms/step - loss: 0.6367 - categorical_accuracy: 0.7428 - val_loss: 0.2314 - val_categorical_accuracy: 0.9115 - lr: 9.7030e-04
Epoch 5/10
768/768 [==============================] - 2s 2ms/step - loss: 0.6073 - categorical_accuracy: 0.7487 - val_loss: 0.2326 - val_categorical_accuracy: 0.9115 - lr: 9.6060e-04
Epoch 6/10
768/768 [================

In [29]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

193/193 [==============================] - 1s 998us/step - loss: 0.1877 - categorical_accuracy: 0.8860
Test loss:0.18765689432621002, Test accuracy:0.8860103487968445


In [30]:
model.export_model()
!ls exported_model

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
INFO:tensorflow:Assets written to: /tmp/tmp_m_hmf89/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp_m_hmf89/saved_model/assets


best_model_weights.data-00000-of-00001	epoch_models		 metadata.json
best_model_weights.index		gesture_recognizer.task
checkpoint				logs


2024-01-20 14:23:09.594858: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-01-20 14:23:09.594892: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-01-20 14:23:09.595310: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp_m_hmf89/saved_model
2024-01-20 14:23:09.596186: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-01-20 14:23:09.596245: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp_m_hmf89/saved_model
2024-01-20 14:23:09.601007: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-01-20 14:23:09.601778: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-01-20 14:23:09.621317: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: 